In [1]:
# get permission to access drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Interpretable ML question: Provide global and local interpretations of a model, either from a previous problem set or from your course project. Please do all three of the following:
1. Permutation feature importance

In [2]:
# The following has been copied from problem set 1: 
# we use the model having frequencies over trigrams ending as a noun as features
import pandas as pd
import numpy as np
datapath = '/content/gdrive/My Drive/Colab Notebooks/data/'
cases = pd.read_csv(datapath + 'cases_metadata.csv')
cases = cases.dropna(subset=['log_cites'])
no_of_cases = 1000
random_indices = list(np.random.choice(cases.shape[0], no_of_cases, replace = False))
caseids = list(cases.iloc[random_indices]["caseid"])

In [3]:
# load the cases
import glob

text = {}
casepath = datapath + 'cases/'
for caseid in caseids:
    names = glob.glob(casepath + "*_" + caseid + ".txt")  # try to find file with particular caseid
    if len(names) > 0:  # if file exists
        filename = names[0]
        file = open(filename, mode = 'r')
        case = filename[len(casepath):-len('.txt')]  # keep year_caseid
        text[case] = file.read()
        file.close()


In [4]:
import spacy
nlp = spacy.load('en', disable=['ner'])
docs = {}
for case in text:
    docs[case] = nlp(text[case])

In [5]:
words = {}
for case in docs:
    words[case] = [token for token in docs[case] if token.is_alpha]

In [6]:
# save part of speech tags for later
pos = {str(word).lower(): word.pos_ for word in words[case] for case in words}

In [7]:
from string import punctuation
translator = str.maketrans('', '', punctuation) 

##################################################
# - we do not remove stopwords, since otherwise the trigrams are less meaningful
# - we do not stem the words, since this might change the part of speech tag (in that case, we cannot correctly find trigrams ending in a noun)
##################################################
def normalize_text(doc):
    tokens = []
    for sent in doc.sents:
        sent = str(sent)
        sent = sent.replace('\r', ' ').replace('\n', ' ')
        lower = sent.lower()  # all lower case
        nopunc = lower.translate(translator)  # remove punctuation
        words = nopunc.split()  # split into tokens
        no_numbers = [w if not w.isdigit() else '#' for w in words]  # normalize numbers
        tokens += no_numbers  # add to list of tokens for this speech
    return tokens

In [8]:
normalized = {}
for case in docs:
    normalized[case] = normalize_text(docs[case])

In [9]:
# make a feature set of all trigrams that end in a noun
import nltk
from nltk import trigrams
ends_in_noun = {}
for case in normalized:
    trigrams = list(nltk.trigrams(normalized[case]))
    ends_in_noun[case] = []
    for trigram in trigrams:
        string = " ".join(trigram)
        if trigram[2] in pos and pos[trigram[2]] == 'NOUN':
            ends_in_noun[case].append(string)

In [10]:
from collections import Counter
frequencies = {}
for case in ends_in_noun:
    to_count = (trigram for trigram in ends_in_noun[case])
    frequencies[case] = Counter(to_count).most_common()

In [11]:
# features: the trigrams that are contained in all documents, plus the top-K most common ones in any of the documents
import math
features = set()
no_of_most_common_trigrams = math.ceil(1000 / no_of_cases)  # this is the K we choose for top-K
intersection = set(map(lambda x: x[0], list(frequencies.values())[0]))  # initially contains all trigrams of arbitrary document
for case in frequencies:
    for trigram in set(map(lambda x: x[0], frequencies[case][0:min(no_of_most_common_trigrams, len(frequencies[case]))])):
        features.add(trigram)
    intersection = intersection.intersection(set(map(lambda x: x[0], frequencies[case])))
features = list(features.union(intersection))

In [12]:
df = pd.DataFrame(columns = ["case"] + features)
for case in frequencies:
    data = {}
    data["case"] = case[5:len(case)]  # now keeping only case id and throwing away year
    for feature in features:
        if feature in dict(frequencies[case]):
            data[feature] = dict(frequencies[case])[feature]
        else:
            data[feature] = 0
    df = df.append(data, ignore_index = True)

for feature in features:
    df[feature] /= np.sqrt(np.var(df[feature]))  # standardize to variance 1

In [13]:
df = df.join(pd.read_csv(datapath + 'case_metadata.csv').set_index('caseid'), on='case')

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
X = df.loc[:, features]
y = df.case_reversed

# We use a gradient boosting classifier
from sklearn.ensemble import GradientBoostingClassifier
boost = GradientBoostingClassifier().fit(X, y)

In [16]:
from eli5.sklearn import PermutationImportance
import eli5
perm = PermutationImportance(boost).fit(X, y)
eli5.show_weights(perm, feature_names = features)

Weight,Feature
0.0127 ± 0.0111,the tax court
0.0105 ± 0.0037,the third circuit
0.0101 ± 0.0038,in the case
0.0081 ± 0.0064,the congressional intent
0.0067 ± 0.0049,in this case
0.0063 ± 0.0024,that the evidence
0.0063 ± 0.0024,the interstate commerce
0.0042 ± 0.0024,goods for commerce
0.0040 ± 0.0038,for the purpose
0.0040 ± 0.0013,federal common law


2. Global surrogate model

In [17]:
from sklearn.linear_model import LassoCV
y_hat = boost.predict(X)
lasso = LassoCV(cv=10).fit(X, y_hat)  # fit surrogate model

In [18]:
from sklearn.metrics import r2_score
y_hat_hat = lasso.predict(X)
r2_score(y_hat, y_hat_hat)  # validate that black box predictions are replicated

0.7173121755828777

In [19]:
from itertools import compress
# list most important features, i.e. with large coefficient (in absolute sense). 
# The threshold 0.02 is somewhat arbitrary, but chosen in such a way that the list of features is not too long
important_features = abs(lasso.coef_) > 0.02
list(compress(features, important_features))

['a jury trial',
 'by any court',
 'on the motion',
 'the congressional intent',
 'of the value',
 'goods for commerce',
 'to the company',
 'the common law',
 'equal civil rights',
 'the texas law',
 'was no consideration',
 'use of interstate',
 'the parties intent',
 'the interstate commerce',
 'in the case',
 'an equitable defense',
 'the third circuit',
 'on interstate commerce',
 'v mitchell supra',
 'the tax court']

3. Local surrogate model using lasso (LIME)

In [20]:
def predict_proba(self, text = 'This is an example text.'): 
    # the first part is feature extraction: finding the text's frequencies over the given trigrams
    docs = nlp(str(text))
    words = [token for token in docs if token.is_alpha]
    pos = {str(word).lower(): word.pos_ for word in words}
    normalized = normalize_text(docs)
    trigrams = list(nltk.trigrams(normalized))
    ends_in_noun = []
    for trigram in trigrams:
        string = " ".join(trigram)
        if trigram[2] in pos and pos[trigram[2]] == 'NOUN':
            ends_in_noun.append(string)
    to_count = (trigram for trigram in ends_in_noun)
    frequencies = Counter(to_count).most_common()
    
    # put these features in a dataframe
    df = pd.DataFrame(columns = features)
    data = {}
    for feature in features:
        if feature in dict(frequencies):
            data[feature] = dict(frequencies)[feature]
        else:
            data[feature] = 0
    df = df.append(data, ignore_index = True)
    
    # use the fitted black box model to predict
    preds = boost.predict_proba(df).reshape(-1, 1)
    p0 = 1 - preds
    return np.hstack((p0, preds))

In [21]:
from eli5.lime import TextExplainer
import random

# The following will be repeated three times for diversity (three random cases will be shown)
case_with_year = random.choice(list(text.keys()))  # choose a random case in the dictionary
case = case_with_year[5:len(case_with_year)]  # keep only the caseid (and throw away the year)
te = TextExplainer().fit(text[case_with_year], predict_proba)
te.show_prediction(targets=[int(df.case_reversed[df.case == case])])

In [22]:
case_with_year = random.choice(list(text.keys()))
case = case_with_year[5:len(case_with_year)]
te.fit(text[case_with_year], predict_proba)
te.show_prediction(targets=[int(df.case_reversed[df.case == case])])

In [23]:
case_with_year = random.choice(list(text.keys()))
case = case_with_year[5:len(case_with_year)]
te.fit(text[case_with_year], predict_proba)
te.show_prediction(targets=[int(df.case_reversed[df.case == case])])